In [1]:
from qiskit.opflow import Z, I, X, Y
from qiskit.opflow import CircuitStateFn, StateFn, CircuitSampler, PauliExpectation, ListOp, TensoredOp

from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector
from qiskit.algorithms.optimizers import ADAM, SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library.n_local import EfficientSU2, RealAmplitudes
from qiskit import Aer, execute

import numpy as np
import matplotlib.pyplot as plt

In [2]:
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

In [3]:
a0 = FermionicOp('-III')
a0_dagger = FermionicOp('+III')
a1 = FermionicOp('I-II')
a1_dagger = FermionicOp('I+II')
a2 = FermionicOp('II-I')
a2_dagger = FermionicOp('II+I')
a3 = FermionicOp('III-')
a3_dagger = FermionicOp('III+')

H = (FermionicOp('-III') + FermionicOp('+III')) @ (FermionicOp('I-II') + FermionicOp('I+II')) @ (FermionicOp('IIII') - 2*(FermionicOp('+III') @ FermionicOp('-III'))) 
H += (FermionicOp('I-II') - FermionicOp('I+II')) @ (FermionicOp('II+I') - FermionicOp('II-I')) @ (FermionicOp('IIII') - 2*(FermionicOp('I+II') @ FermionicOp('I-II'))) 
H += (FermionicOp('IIII') - 2*(FermionicOp('II+I') @ FermionicOp('II-I'))) @ (FermionicOp('IIII') - 2*(FermionicOp('III+') @ FermionicOp('III-'))) 

print(H)

mapper = JordanWignerMapper()

print(mapper.map(H))
H = mapper.map(H)

Fermionic Operator
register length=4, number terms=20
  (1+0j) * ( --II )
+ (1+0j) * ( +-II )
+ (1+0j) * ( -+II )
+ (1+0j) * ( ++II )
+ (-2+0j) * ( --II )
+ (-2+0j) * ( -+II )
+ (1+0j) * ( I-+I )
+ (-1+0j) * ( I++I )
+ (-1+0j) * ( I--I )
+ (1+0j) * ( ...
1.0 * ZZII
+ 1.0 * IYYI
+ 1.0 * IIXX


<ipython-input-3-754760bbdfb7>:1: UserWarning: The default value for `display_format` will be changed from 'dense' to 'sparse' in version 0.3.0. Once that happens, you must specify display_format='dense' directly.
  a0 = FermionicOp('-III')


In [4]:
n_qubits = 4
params = ParameterVector('θ',(n_qubits*4))

qr = QuantumRegister(n_qubits)
ansatz = QuantumCircuit(qr)

       
ansatz.compose(EfficientSU2(4, reps=1, insert_barriers=True).decompose(), inplace=True)
ansatz.draw()

┌──────────┐┌──────────┐ ░                                ░  ┌──────────┐»
q0_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├─░───■────■────■──────────────────░──┤ Ry(θ[8]) ├»
      ├──────────┤├──────────┤ ░ ┌─┴─┐  │    │                  ░  ├──────────┤»
q0_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├──┼────┼────■────■────────░──┤ Ry(θ[9]) ├»
      ├──────────┤├──────────┤ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░ ┌┴──────────┤»
q0_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░─┤ Ry(θ[10]) ├»
      ├──────────┤├──────────┤ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ├───────────┤»
q0_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─░───────────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(θ[11]) ├»
      └──────────┘└──────────┘ ░           └───┘     └───┘└───┘ ░ └───────────┘»
«      ┌───────────┐
«q0_0: ┤ Rz(θ[12]) ├
«      ├───────────┤
«q0_1: ┤ Rz(θ[13]) ├
«      ├───────────┤
«q0_2: ┤ Rz(θ[14]) ├
«      ├───────────┤
«q0_3: ┤ Rz(θ[15]) ├
«      └───────────┘

In [5]:
from qiskit.providers.aer import AerSimulator

sim = AerSimulator()

from qiskit.algorithms.optimizers import SPSA

optimizer = SPSA(maxiter=300)

from qiskit.algorithms import VQE as VQE_q

vqe = VQE_q(
    ansatz=ansatz,
    optimizer=optimizer,
    quantum_instance=sim  # use qiskit.utils.QuantumInstance for error mitigation and other features
)

vqe_result = vqe.compute_minimum_eigenvalue(H)

In [6]:
print(vqe_result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 600,
    'eigenstate': {   '0001': 0.15625,
                      '0010': 0.16237976320958225,
                      '0100': 0.6952686081652184,
                      '0111': 0.6817945071647321,
                      '1000': 0.03125},
    'eigenvalue': (-2.23828125+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[7]): -6.395952848173277,
                              ParameterVectorElement(θ[4]): 2.250509332217714,
                              ParameterVectorElement(θ[1]): 0.4324056782064047,
                              ParameterVectorElement(θ[6]): -3.4016515185029284,
                              ParameterVectorElement(θ[11]): 6.226143981483694,
                              ParameterVectorElement(θ[3]): -6.218365322324229,
                              ParameterVectorElement(θ[13]): -4.321134059757356,
                              ParameterVectorElement(θ[15]): 1.7460062016947542,
                      

In [7]:
from qiskit.algorithms import Eigensolver

es = Eigensolver()

es.compute_eigenvalues(H)

TypeError: Can't instantiate abstract class Eigensolver with abstract methods compute_eigenvalues